In [0]:
pip install confluent-kafka supabase

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 729.3/729.3 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 38.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.12/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-056598c6-c2e1-47ee-89c2-0ee0df7456cf
    Can't uninstall 'typing_extensions'. No files were found to uninstall.
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.27.2
    Not uninstalling pydantic-core at /databricks/python3/lib/python3.12/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-056598c6-c2e1-47ee-89c2-0ee0df7456cf
    Can't uninstall 'pydantic_core'. No f

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS invistis;
CREATE SCHEMA IF NOT EXISTS invistis.datalake;
CREATE VOLUME IF NOT EXISTS invistis.datalake.raw;


In [0]:
SUPABASE_HOST = "https://dtbhervknmfsgrfzbkxm.supabase.co" # ton host Supabase
SUPABASE_PORT = "5432"
SUPABASE_DB   = "postgres"
SUPABASE_USER = "postgres"
SUPABASE_PASSWORD = "#Ex9_x3kR7/k7NJ"

supabase_url = f"jdbc:postgresql://{SUPABASE_HOST}:{SUPABASE_PORT}/{SUPABASE_DB}"

supabase_props = {
    "user": SUPABASE_USER,
    "password": SUPABASE_PASSWORD,
    "driver": "org.postgresql.Driver",
    "sslmode": "require"
}

In [0]:
CONFLUENT_BOOTSTRAP = "pkc-921jm.us-east-2.aws.confluent.cloud:9092"
CONFLUENT_API_KEY   = "JWVBQ7RG25AVCHHY"
CONFLUENT_SECRET    = "cflt9aP2o4QFY2CAugnhB/J/MMhEIDYtyGdj3Gu6dcFih68+sqYqyvBvzj8ABd4g"

CHECKPOINT_PATH = "/Volumes/invistis/datalake/raw/checkpoints"
DATA_PATH       = "/Volumes/invistis/datalake/raw/enriched"

kafka_options = {
    "kafka.bootstrap.servers": CONFLUENT_BOOTSTRAP,
    "kafka.sasl.mechanism": "PLAIN",
    "kafka.security.protocol": "SASL_SSL",
    "kafka.sasl.jaas.config": f'kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username="{CONFLUENT_API_KEY}" password="{CONFLUENT_SECRET}";',
    "startingOffsets": "earliest",
    "failOnDataLoss": "false"
}
print("✅ Config OK")

✅ Config OK


In [0]:

from pyspark.sql.functions import *
from pyspark.sql.types import *

trades_schema = StructType([
    StructField("symbol", StringType()),
    StructField("price", DoubleType()),
    StructField("quantity", DoubleType()),
    StructField("timestamp", LongType()),
    StructField("trade_id", StringType()),
])

trades_df = (spark.readStream
    .format("kafka")
    .options(**kafka_options)
    .option("subscribe", "trades_topic")
    .load()
    .select(from_json(col("value").cast("string"), trades_schema).alias("data"), "timestamp")
    .select("data.*", col("timestamp").alias("kafka_ts"))
    .withColumn("event_time", to_timestamp(col("timestamp") / 1000))
    .withWatermark("event_time", "5 minutes")
)
print("✅ Trades stream OK")

✅ Trades stream OK


In [0]:
news_schema = StructType([
    StructField("title", StringType()),
    StructField("description", StringType()),
    StructField("source", StringType()),
    StructField("published_at", StringType()),
    StructField("sentiment", DoubleType()),
])

news_df = (spark.readStream
    .format("kafka")
    .options(**kafka_options)
    .option("subscribe", "news_topic")
    .load()
    .select(from_json(col("value").cast("string"), news_schema).alias("data"), "timestamp")
    .select("data.*", col("timestamp").alias("kafka_ts"))
    .withColumn("event_time", to_timestamp(col("published_at")))
    .withColumn("symbol", lit("BTCUSDT"))
    .withWatermark("event_time", "5 minutes")
)
print("✅ News stream OK")

✅ News stream OK


In [0]:
from pyspark.sql.functions import *

# Stream-to-Stream Join avec fenêtre plus large
joined_df = trades_df.alias("t").join(
    news_df.alias("n"),
    expr("""
        t.symbol = n.symbol AND
        t.event_time BETWEEN n.event_time - INTERVAL 24 HOURS
        AND n.event_time + INTERVAL 24 HOURS
    """),
    "left"
).select(
    col("t.symbol"),
    col("t.price"),
    col("t.quantity"),
    col("t.event_time").alias("trade_time"),
    col("n.title").alias("news_title"),
    col("n.sentiment"),
    col("n.event_time").alias("news_time"),
    window(col("t.event_time"), "5 minutes").alias("window_5min")
)

# Paths
CHECKPOINT_PATH = "/Volumes/invistis/datalake/raw/checkpoints"
DATA_PATH = "/Volumes/invistis/datalake/raw/enriched"

# Sink 1 : Delta Lake
query_lake = (joined_df.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", f"{CHECKPOINT_PATH}/lake2")
    .option("path", DATA_PATH)
    .trigger(availableNow=True)
    .start()
)

query_lake.awaitTermination()
print("✅ Done!")

✅ Done!


In [0]:
# Supprimer l'ancienne table pour repartir propre
dbutils.fs.rm("/Volumes/invistis/datalake/raw/enriched", recurse=True)
dbutils.fs.rm("/Volumes/invistis/datalake/raw/checkpoints/lake_v3", recurse=True)
print("✅ Ancienne table supprimée")

✅ Ancienne table supprimée


In [0]:
query_lake = (joined_df
    .writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", f"{CHECKPOINT_PATH}/lake_v4")
    .option("path", DATA_PATH)
    .foreachBatch(lambda df, epoch_id: 
        df.select(
            col("symbol"),
            col("price"),
            col("quantity"),
            col("trade_time"),
            col("news_title"),
            col("sentiment"),
            col("news_time"),
            window(col("trade_time"), "5 minutes").getField("start").alias("window_start"),
            window(col("trade_time"), "5 minutes").getField("end").alias("window_end")
        ).write.format("delta").mode("append").save(DATA_PATH)
    )
    .trigger(availableNow=True)
    .start()
)

query_lake.awaitTermination()
print("✅ Stream-to-Stream Join + Delta Lake OK !")

✅ Stream-to-Stream Join + Delta Lake OK !


In [0]:
# Sauvegarder avec overwriteSchema
joined_batch.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(DATA_PATH)

# Afficher
df = spark.read.format("delta").load(DATA_PATH)
print(f"✅ Total rows: {df.count()}")
display(df.limit(20))

✅ Total rows: 3993


symbol,price,quantity,trade_time,news_title,sentiment,news_time,window_5min
BTCUSDT,63224.1,2.0E-4,2026-02-24T12:11:28.796Z,null,null,null,"List(2026-02-24T12:10:00.000Z, 2026-02-24T12:15:00.000Z)"
BTCUSDT,63224.1,1.7E-4,2026-02-24T12:11:29.935Z,null,null,null,"List(2026-02-24T12:10:00.000Z, 2026-02-24T12:15:00.000Z)"
BTCUSDT,63224.1,0.00326,2026-02-24T12:11:29.935Z,null,null,null,"List(2026-02-24T12:10:00.000Z, 2026-02-24T12:15:00.000Z)"
BTCUSDT,63224.1,0.02086,2026-02-24T12:11:29.935Z,null,null,null,"List(2026-02-24T12:10:00.000Z, 2026-02-24T12:15:00.000Z)"
BTCUSDT,63224.1,1.7E-4,2026-02-24T12:11:31.727Z,null,null,null,"List(2026-02-24T12:10:00.000Z, 2026-02-24T12:15:00.000Z)"
BTCUSDT,63224.1,8.0E-5,2026-02-24T12:11:31.727Z,null,null,null,"List(2026-02-24T12:10:00.000Z, 2026-02-24T12:15:00.000Z)"
BTCUSDT,63224.1,6.9E-4,2026-02-24T12:11:31.727Z,null,null,null,"List(2026-02-24T12:10:00.000Z, 2026-02-24T12:15:00.000Z)"
BTCUSDT,63224.1,4.8E-4,2026-02-24T12:11:31.727Z,null,null,null,"List(2026-02-24T12:10:00.000Z, 2026-02-24T12:15:00.000Z)"
BTCUSDT,63224.1,4.7E-4,2026-02-24T12:11:31.727Z,null,null,null,"List(2026-02-24T12:10:00.000Z, 2026-02-24T12:15:00.000Z)"
BTCUSDT,63224.11,0.00764,2026-02-24T12:11:32.176Z,null,null,null,"List(2026-02-24T12:10:00.000Z, 2026-02-24T12:15:00.000Z)"


In [0]:
import requests
import json
import numpy as np
import pandas as pd

def clean_record(row):
    record = {}
    for k, v in row.items():
        if v is None:
            record[k] = None
        elif isinstance(v, float) and np.isnan(v):
            record[k] = None
        elif isinstance(v, pd.NaT.__class__) or v is pd.NaT:
            record[k] = None
        elif hasattr(v, 'isoformat'):  # datetime/timestamp → string ISO
            record[k] = v.isoformat()
        elif isinstance(v, np.integer):
            record[k] = int(v)
        elif isinstance(v, np.floating):
            record[k] = None if np.isnan(v) else float(v)
        else:
            record[k] = v
    return record

# Remplacer tous les NaT par None AVANT la boucle
pdf_clean = pdf.where(pdf.notna(), None)

records = [clean_record(row) for _, row in pdf_clean.iterrows()]

# Vérification sur 1 record
print("Sample record:", json.dumps(records[0], default=str))

# Envoi
BATCH_SIZE = 500
success = 0

for i in range(0, len(records), BATCH_SIZE):
    batch = records[i:i+BATCH_SIZE]
    response = requests.post(
        f"{SUPABASE_URL}/rest/v1/enriched_trades",
        headers=headers,
        data=json.dumps(batch)
    )
    if response.status_code in [200, 201]:
        success += len(batch)
        print(f"✅ Batch {i//BATCH_SIZE + 1} envoyé ({success}/{len(records)})")
    else:
        print(f"❌ Erreur: {response.status_code} - {response.text}")

print(f"✅ Total envoyé : {success} lignes !")

Sample record: {"symbol": "BTCUSDT", "price": 63224.1, "quantity": 0.0002, "trade_time": "2026-02-24T12:11:28.796000", "news_title": null, "sentiment": null, "news_time": null, "window_start": "2026-02-24T12:10:00", "window_end": "2026-02-24T12:15:00"}
✅ Batch 1 envoyé (500/3993)
✅ Batch 2 envoyé (1000/3993)
✅ Batch 3 envoyé (1500/3993)
✅ Batch 4 envoyé (2000/3993)
✅ Batch 5 envoyé (2500/3993)
✅ Batch 6 envoyé (3000/3993)
✅ Batch 7 envoyé (3500/3993)
✅ Batch 8 envoyé (3993/3993)
✅ Total envoyé : 3993 lignes !
